In [1]:
%reset
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import defmod as dm
import multimodule_usefulfunctions as mm 
import hamiltonian_multishape as ham
import kernels as ker
import model_MultiShapeCirclesTranslation as model

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
## define reference and template shape
r = [3,2]
origin1 = [[-3,0], [6,6]]
origin2 = [[0,3], [6,4]]
nb_points = [10,7]

source = mm.multipleCircles(origin1, r ,nb_points)
target = mm.multipleCircles(origin2, r ,nb_points)

In [3]:
## Define Deformation Modules
dim = 2
nb_shapes = 2

# Module 1: Translation of first circle
sigma1 = 10
trans1 = dm.deformationmodules.Translations(dim, 1, sigma1)

# Module 2: Translation of second circle
sigma2 = 10
trans2 = dm.deformationmodules.Translations(dim,1, sigma2)

# Module 3: Background Module
sigma3 = 1.5
nb_pts = nb_points[0] + nb_points[1]
background = dm.deformationmodules.Translations(dim, nb_pts, sigma3)


module_list = [trans1, trans2, background]
dim_controls = dim*(1+1+nb_pts)
gd_list = [source[0], source[1], torch.cat([source[0], source[1]],0)]

In [4]:
target_list = [*target, torch.cat([target[0], target[1]])]

In [5]:
sigma_list = [sigma1, sigma2, sigma3]
z = [mm.computeCenter(source[0]), mm.computeCenter(source[1])]

#initialize gd and momentum
mom_tmp0 = np.zeros([nb_points[0], dim])
mom_tmp1 = np.zeros([nb_points[1], dim])
gd0_list = [source[0], source[1], torch.cat([source[0], source[1]],0)]
mom0_list = [torch.tensor(mom_tmp0.copy(), requires_grad=True, dtype=torch.float32), torch.tensor(mom_tmp1.copy(),requires_grad=True, dtype=torch.float32), torch.tensor(np.zeros(gd_list[2].shape),requires_grad=True, dtype=torch.float32)]

# Constraints (identity)
Constr1 = torch.cat([torch.eye(nb_points[0]), torch.zeros([nb_points[0], nb_points[1]]), -torch.eye(nb_points[0]), torch.zeros([nb_points[0], nb_points[1]])], 1)
Constr2 = torch.cat([torch.zeros([nb_points[1], nb_points[0]]), torch.eye(nb_points[1]), torch.zeros([nb_points[1], nb_points[0]]), -torch.eye(nb_points[1]),], 1)
Constr = [Constr1, Constr2]

# reduced Hamiltonian 
dim_control = [[1,2], [1,2], [nb_pts,2]]
H = ham.Hamiltonian_Multishape(module_list, dim_control)

In [6]:
# set parameters
reg_param = 10**-2 # regularisation parameter

gd0_tensor = torch.cat([gd0_list[0], gd0_list[1], gd0_list[2]],0)
mom0_tensor = torch.cat([mom0_list[0], mom0_list[1], mom0_list[2]],0)
    
EnergyFunctional = model.EnergyFunctional(module_list, H, Constr, target_list, sigma_list, dim, reg_param)
gradE = torch.autograd.grad(EnergyFunctional.energy_tensor(gd0_tensor, mom0_tensor), mom0_tensor)
    
# do registration
X = [gd0_tensor, mom0_tensor]
X = model.gradientdescent(EnergyFunctional.energy_tensor, EnergyFunctional.gradE, X)

cost: tensor(0., grad_fn=<AddBackward0>)
attach: tensor(33.1301, grad_fn=<AddBackward0>)
gamma: 0.01
total: tensor(33.1301, grad_fn=<AddBackward0>)
cost: tensor(0., grad_fn=<AddBackward0>)
attach: tensor(33.1301, grad_fn=<AddBackward0>)
gamma: 0.01
total: tensor(33.1301, grad_fn=<AddBackward0>)
cost: tensor(0., grad_fn=<AddBackward0>)
attach: tensor(33.1301, grad_fn=<AddBackward0>)
gamma: 0.01
total: tensor(33.1301, grad_fn=<AddBackward0>)
 iter : 0  ,total energy: 33.1301155090332
cost: tensor(0.0021, grad_fn=<AddBackward0>)
attach: tensor(33.3014, grad_fn=<AddBackward0>)
gamma: 0.01
total: tensor(33.3014, grad_fn=<AddBackward0>)
tensor(1, dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
tensor(0, dtype=torch.uint8)
False
convergence True
cost: tensor(0.0021, grad_fn=<AddBackward0>)
attach: tensor(33.3014, grad_fn=<AddBackward0>)
gamma: 0.01
total: tensor(33.3014, grad_fn=<AddBackward0>)
 iter : 1  ,total energy: 33.30139923095703


In [7]:
mom = EnergyFunctional.tensor2list(X[1])
reg_param*EnergyFunctional.cost(gd0_list, mom)+ EnergyFunctional.attach(gd0_list, mom)

tensor(63.1251, grad_fn=<AddBackward0>)

In [8]:
# check if autograd is correct

#torch.autograd.gradcheck(EnergyFunctional.energy_tensor, (gd0_tensor, mom0_tensor), eps=1e-2, atol=1e-2, raise_exception=True)


In [9]:
# check if gradient wrt controls is zero
mom = [torch.randn(mom0_list[0].shape, requires_grad=True), torch.randn(mom0_list[1].shape,requires_grad=True), torch.randn(mom0_list[2].shape,requires_grad=True)]
gd, _, control = EnergyFunctional.shoot(gd0_list, mom)
i = 0
z_list = [mm.computeCenter(gd[i][0]), mm.computeCenter(gd[i][1]), gd[i][2]]
torch.autograd.grad(EnergyFunctional.cost(z_list, control[i]), control[i])

(tensor([[-0.2869, -0.6388]]),
 tensor([[-0.5057, -0.2373]]),
 tensor([[-0.2622, -0.5838],
         [-0.2622, -0.5838],
         [-0.2622, -0.5838],
         [-0.2622, -0.5838],
         [-0.2622, -0.5838],
         [-0.2622, -0.5838],
         [-0.2622, -0.5838],
         [-0.2622, -0.5838],
         [-0.2622, -0.5838],
         [-0.2622, -0.5838],
         [-0.4859, -0.2280],
         [-0.4859, -0.2280],
         [-0.4859, -0.2280],
         [-0.4859, -0.2280],
         [-0.4859, -0.2280],
         [-0.4859, -0.2280],
         [-0.4859, -0.2280]]))

In [10]:
for i in range(len(control)):
    z_list = [mm.computeCenter(gd[i][0]), mm.computeCenter(gd[i][1]), gd[i][2]]
    print(EnergyFunctional.cost(z_list, control[i]))
control
# the control is constant over time, does that make sense?

tensor(2.4707, grad_fn=<AddBackward0>)
tensor(2.4707, grad_fn=<AddBackward0>)
tensor(2.4707, grad_fn=<AddBackward0>)
tensor(2.4707, grad_fn=<AddBackward0>)
tensor(2.4707, grad_fn=<AddBackward0>)
tensor(2.4707, grad_fn=<AddBackward0>)
tensor(2.4707, grad_fn=<AddBackward0>)
tensor(2.4707, grad_fn=<AddBackward0>)
tensor(2.4707, grad_fn=<AddBackward0>)
tensor(2.4707, grad_fn=<AddBackward0>)
tensor(2.4707, grad_fn=<AddBackward0>)


[[tensor([[-0.2869, -0.6388]], grad_fn=<MmBackward>),
  tensor([[-0.5057, -0.2373]], grad_fn=<MmBackward>),
  tensor([[-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.3129, -0.1469],
          [-0.3129, -0.1469],
          [-0.3129, -0.1469],
          [-0.3129, -0.1469],
          [-0.3129, -0.1469],
          [-0.3129, -0.1469],
          [-0.3129, -0.1469]], grad_fn=<AddBackward0>)],
 [tensor([[-0.2869, -0.6388]], grad_fn=<MmBackward>),
  tensor([[-0.5057, -0.2373]], grad_fn=<MmBackward>),
  tensor([[-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
          [-0.1818, -0.4048],
       